<a href="https://colab.research.google.com/github/TaisGomess/nasofibroscopia-prototipo/blob/main/Nasofibroscopia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow keras opencv-python matplotlib pandas

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

base_dir = '/content/drive/MyDrive/Nasofibroscopia'
classes = ['normal', 'polipo', 'inflamacao']

In [8]:
import cv2
import numpy as np
import os

img_size = 128  # tamanho padrão da imagem

data = []
labels = []

for idx, classe in enumerate(classes):
    path = os.path.join(base_dir, classe)
    print(f"Processing class: {classe} from path: {path}") # Add print statement
    if not os.path.exists(path): # Add path existence check
        print(f"Error: Directory not found at {path}")
        continue
    for img_name in os.listdir(path):
        img_path = os.path.join(path, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (img_size, img_size))
            data.append(img)
            labels.append(idx)
        else: # Add check for failed image loading
            print(f"Warning: Could not load image {img_path}")


data = np.array(data) / 255.0  # normalizando
labels = np.array(labels)

print(f"Loaded {len(data)} images.") # Print number of loaded images

Processing class: normal from path: /content/drive/MyDrive/Nasofibroscopia/normal
Processing class: polipo from path: /content/drive/MyDrive/Nasofibroscopia/polipo
Processing class: inflamacao from path: /content/drive/MyDrive/Nasofibroscopia/inflamacao
Loaded 0 images.


In [6]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

labels_cat = to_categorical(labels, num_classes=len(classes))

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
x = Flatten()(base_model.output)
x = Dense(64, activation='relu')(x)
output = Dense(len(classes), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

for layer in base_model.layers:
    layer.trainable = False  # mantém o pré-treinado

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [10]:
print("Total de imagens carregadas:", len(data))
print("Total de rótulos:", len(labels))
print("Classes encontradas:", set(labels))

Total de imagens carregadas: 0
Total de rótulos: 0
Classes encontradas: set()


In [11]:
!ls "/content/drive/MyDrive/Nasofibroscopia"

inflamacao  normal  polipo


In [12]:
!ls "/content/drive/MyDrive/Nasofibroscopia/normal"

normal.jpg


In [13]:
valid_exts = ('.jpg', '.jpeg', '.png', '.bmp')

data = []
labels = []

for idx, classe in enumerate(classes):
    path = os.path.join(base_dir, classe)
    if not os.path.exists(path):
        print(f"Aviso: Pasta {classe} não encontrada, pulando...")
        continue

    for img_name in os.listdir(path):
        if img_name.lower().endswith(valid_exts):
            img_path = os.path.join(path, img_name)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (img_size, img_size))
                data.append(img)
                labels.append(idx)

print(f"Imagens carregadas: {len(data)}")

Imagens carregadas: 3


In [15]:
print("Quantidade de imagens:", len(data))
print("Quantidade de labels:", len(labels))
print("Labels:", labels)

Quantidade de imagens: 3
Quantidade de labels: 3
Labels: [0, 1, 2]


In [16]:
from tensorflow.keras.utils import to_categorical

if len(labels) > 0:
    labels_cat = to_categorical(labels, num_classes=len(classes))
else:
    raise ValueError("Nenhuma imagem foi carregada, adicione imagens nas pastas!")

In [18]:
import numpy as np

data = np.array(data, dtype="float32") / 255.0  # normaliza e vira array numpy
print("Shape do dataset:", data.shape)

Shape do dataset: (3, 128, 128, 3)


In [19]:
print("Imagens:", len(data), "Labels:", len(labels))

Imagens: 3 Labels: 3


In [20]:
from tensorflow.keras.utils import to_categorical

labels_cat = to_categorical(labels, num_classes=len(classes))
print("Shape dos labels:", labels_cat.shape)

Shape dos labels: (3, 3)


In [21]:
if len(data) > 1 and len(labels_cat) > 1:
    model.fit(data, labels_cat, epochs=3, batch_size=1)
else:
    print("⚠️ Adicione pelo menos 2 imagens por classe para treinar corretamente.")

Epoch 1/3
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 54ms/step - accuracy: 0.5417 - loss: 6.5543
Epoch 2/3
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.8333 - loss: 0.7755
Epoch 3/3
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 1.0000 - loss: 0.0210
